In [1]:
!mkdir data
!wget https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip -O ./data/chgk.zip

--2022-04-06 17:24:04--  https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.72.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.72.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/s4qj0fpsn378m2i/chgk.zip [following]
--2022-04-06 17:24:04--  https://www.dropbox.com/s/raw/s4qj0fpsn378m2i/chgk.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc697226fd1b3cae15c35ae04b99.dl.dropboxusercontent.com/cd/0/inline/Bi4TwOyWPhWzHpCPOaEBHdq0RMvZn6eyKZ8IoR33QqlyvF0z2W8pefHHzHYNE-TLp9FAqL0vyx9ysTNAVpuwjmgDEjn0AGEICqSv6mukiCtQ3oT9Wj-z4UmAWXP7WwRAr6dSyz-9zeQdz0FIUcHBpos3RhOR76KEPjc9ApGC5nf8eA/file# [following]
--2022-04-06 17:24:04--  https://uc697226fd1b3cae15c35ae04b99.dl.dropboxusercontent.com/cd/0/inline/Bi4TwOyWPhWzHpCPOaEBHdq0RMvZn6eyKZ8IoR33QqlyvF0z2W8pefHHzHYNE-TLp9FAqL0vyx9ys

In [2]:
import pandas as pd
import utils

In [3]:
DATA_PATH = "data/"

In [4]:
utils.unzip(DATA_PATH + "chgk.zip", DATA_PATH)

In [5]:
players = (pd.DataFrame.from_dict(pd.read_pickle(DATA_PATH + "players.pkl")).T).set_index("id")
results = pd.read_pickle(DATA_PATH + "results.pkl")
tournaments = (pd.DataFrame.from_dict(pd.read_pickle(DATA_PATH + "tournaments.pkl")).T).set_index("id")

/home/maksim.stsiapura/Banch/MADE/made-advanced-ml-2022-hw/py3ve/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/home/maksim.stsiapura/Banch/MADE/made-advanced-ml-2022-hw/py3ve/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [6]:
players.head()

,name,patronymic,surname
id,,,
1,Алексей,None,Абабилов
10,Игорь,,Абалов
11,Наталья,Юрьевна,Абалымова
12,Артур,Евгеньевич,Абальян
13,Эрик,Евгеньевич,Абальян


In [7]:
tournaments.head()

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [8]:
tournament_results = []

for (tournament_id, tournament_result) in results.items():
    for team_result in tournament_result:
        if "mask" in team_result and team_result["mask"] != None:
            if "teamMembers" in team_result and team_result["teamMembers"]:
                mask = team_result["mask"]
                team_members = team_result["teamMembers"]
                for member in team_members:
                    tournament_results.append({
                        "tournament_id" : tournament_id,
                        "player_id" : member["player"]["id"],
                        "mask" : mask
                    })

In [9]:
tournament_results = pd.DataFrame(tournament_results)

In [10]:
tournament_results.head()

,tournament_id,player_id,mask
0,22,1560,0111011101101110001101110011111111110011111100...
1,22,2935,0111011101101110001101110011111111110011111100...
2,22,3270,0111011101101110001101110011111111110011111100...
3,22,4878,0111011101101110001101110011111111110011111100...
4,22,18935,0111011101101110001101110011111111110011111100...


In [11]:
tournament_results.to_csv(DATA_PATH + "tournament_results.csv")